In [1]:
import os 
import numpy as np
import pandas as pd
from utils.utils import *
import nibabel as nib

In [2]:
model_path = os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_inversed.npy')
model = np.load(model_path, allow_pickle=True)
model.shape

(33797, 9)

In [22]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""


columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)
    for j, roi in enumerate(rois.keys()):
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
       
        scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, j] = scores_per_roi_lh
        results[1, i, j] = scores_per_roi_rh

results[:, 0]
        
        



(2, 8, 12, 12)


array([[[ 3.32431260e-02,  2.62957198e-02,  2.17434852e-02,
          1.20587117e-02,  8.76861622e-03,  8.56351291e-03,
         -3.67543457e-03, -1.10208988e-03, -3.66743731e-04,
         -2.47612027e-03, -1.82840276e-03, -2.80433248e-04],
        [ 2.55351658e-02,  2.21965043e-02,  2.06439289e-02,
          6.24784098e-03,  3.52547123e-03, -3.03241147e-04,
         -4.40239402e-03, -1.69117053e-03,  4.25234715e-03,
         -1.17467085e-03,  4.02479518e-04,  6.08951347e-04],
        [ 2.20140192e-02,  2.59257829e-02,  2.71559116e-02,
          1.82983251e-02,  7.54301454e-03,  8.98253225e-03,
         -2.11239159e-03, -7.41370614e-04,  4.50432202e-03,
         -6.70168814e-04, -2.55327957e-03, -7.47172012e-04],
        [ 2.26664169e-02,  2.79696826e-02,  2.95757549e-02,
          1.19209325e-02,  6.61870452e-03, -6.75151223e-04,
         -3.45920152e-03, -8.94703468e-04,  1.13014056e-02,
          2.81889158e-04,  1.06072891e-03,  1.70517097e-03],
        [ 1.89997692e-02,  2.7532850

In [24]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [25]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.006159,0.013197,0.011803,0.006188,-0.004491,-0.008362,-0.007092,-0.006327,0.000516,-0.004508,-0.000158,-0.000376,V2
V2,0.030054,0.026462,0.023849,0.001388,-0.002515,-0.006414,-0.006345,-0.004847,0.010666,-0.000587,0.000542,0.000373,V1
V3,0.019979,0.035076,0.029437,0.017708,-0.002298,-0.005372,-0.004818,-0.004145,0.011774,0.003130,0.002348,-0.000654,V2
hV4,0.023744,0.034524,0.026778,0.002219,-0.002451,-0.006956,-0.006235,-0.004054,0.024719,0.004418,0.005563,0.002178,V2
VO-1,0.016419,0.028426,0.031096,0.020774,-0.000293,-0.003584,-0.002483,-0.002077,0.016607,0.005119,0.003762,-0.000464,V3
VO-2,0.021211,0.030611,0.031972,0.004618,0.000096,-0.005986,-0.005880,-0.003701,0.031237,0.008778,0.009355,0.003846,V3
PHC-1,0.005832,0.018630,0.022910,0.040554,0.006223,-0.001228,-0.001446,-0.001999,0.011449,0.002021,0.012251,0.003487,hV4
PHC-2,0.002161,0.005589,0.008048,0.008090,0.016420,0.036903,0.048406,0.049857,0.006419,0.005833,0.111117,0.062498,TO-1
LO-1,0.003026,0.004592,0.005773,0.001612,0.009269,0.027603,0.036078,0.047247,0.003043,0.001315,0.052409,0.027868,TO-1
LO-2,0.001889,0.004418,0.008331,0.003766,0.021680,0.040176,0.068090,0.077517,0.007792,0.010345,0.095046,0.065087,TO-1


In [14]:
results.shape

(2, 8, 12, 12)

In [15]:
results[0, 0, 0]

array([-0.05333696, -0.0300514 , -0.01556237, -0.00196099, -0.01018307,
       -0.01066287, -0.00275734, -0.00065352,  0.0001455 , -0.0048377 ,
       -0.01372858, -0.00821951])

In [3]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)

    fits_subj = np.zeros((len(rois.keys()), n_voxels, len(columns)))
    for j, roi in enumerate(rois.keys()):

        



        
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
        
        fits_subj[j] = fits_matrix
       
       # scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
       # scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

      #  results[0, i, j] = scores_per_roi_lh
       # results[1, i, j] = scores_per_roi_rh\
    print(np.mean(fits_subj[0, :cutoff[0],5][belongs[:cutoff[0]] == 'V1']))
    for k, roi in enumerate(rois.keys()):
        scores_per_roi_lh = np.array([np.mean(fits_subj[k, :cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_subj[k, cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, k] = scores_per_roi_lh
        results[1, i, k] = scores_per_roi_rh
    


(2, 8, 12, 12)
0.033243126006174385
0.03127391114635417
-0.005450302514527701
0.0154336386515519
0.01522661283911032
0.00939469784049114
-0.00072955255971458
0.01313440257321492


In [27]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [25]:
subj01_results_rh = pd.DataFrame(results[1,0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.006159,0.013197,0.011803,0.006188,-0.004491,-0.008362,-0.007092,-0.006327,0.000516,-0.004508,-0.000158,-0.000376,V2
V2,0.030054,0.026462,0.023849,0.001388,-0.002515,-0.006414,-0.006345,-0.004847,0.010666,-0.000587,0.000542,0.000373,V1
V3,0.019979,0.035076,0.029437,0.017708,-0.002298,-0.005372,-0.004818,-0.004145,0.011774,0.003130,0.002348,-0.000654,V2
hV4,0.023744,0.034524,0.026778,0.002219,-0.002451,-0.006956,-0.006235,-0.004054,0.024719,0.004418,0.005563,0.002178,V2
VO-1,0.016419,0.028426,0.031096,0.020774,-0.000293,-0.003584,-0.002483,-0.002077,0.016607,0.005119,0.003762,-0.000464,V3
VO-2,0.021211,0.030611,0.031972,0.004618,0.000096,-0.005986,-0.005880,-0.003701,0.031237,0.008778,0.009355,0.003846,V3
PHC-1,0.005832,0.018630,0.022910,0.040554,0.006223,-0.001228,-0.001446,-0.001999,0.011449,0.002021,0.012251,0.003487,hV4
PHC-2,0.002161,0.005589,0.008048,0.008090,0.016420,0.036903,0.048406,0.049857,0.006419,0.005833,0.111117,0.062498,TO-1
LO-1,0.003026,0.004592,0.005773,0.001612,0.009269,0.027603,0.036078,0.047247,0.003043,0.001315,0.052409,0.027868,TO-1
LO-2,0.001889,0.004418,0.008331,0.003766,0.021680,0.040176,0.068090,0.077517,0.007792,0.010345,0.095046,0.065087,TO-1


In [11]:
results_both_hemis = np.mean(results, axis=0)
for i in range(results_both_hemis.shape[0]):
    np.fill_diagonal(results_both_hemis[i, 1:, 1:], -1)
results_both_hemis.shape

(8, 12, 12)

In [28]:
subj01_results = pd.DataFrame(results_both_hemis[6], columns=rois.keys(), index=rois.keys())
subj01_results["best_roi"]  = subj01_results.idxmax(axis=1)
subj01_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.001176,0.004231,0.001043,-0.000333,0.011703,0.005409,-0.000020,-0.000063,-0.005687,-0.000601,-0.000003,0.001350,VO-1
V2,-0.010719,-1.000000,-0.010463,-0.008932,-0.003999,-0.004042,-0.000269,0.001459,-0.028010,-0.002114,0.000067,0.002738,TO-2
V3,-0.006759,-0.005656,-1.000000,-0.011352,0.005307,0.001299,-0.001266,0.000349,-0.024541,-0.001103,0.001504,0.004961,VO-1
hV4,-0.022841,-0.031197,-0.034737,-1.000000,-0.010521,-0.009336,-0.003491,-0.002342,-0.057400,-0.006178,-0.002347,0.001282,TO-2
VO-1,-0.006264,-0.008408,-0.012551,-0.009309,-1.000000,-0.000036,0.000295,0.001013,-0.025453,-0.002593,-0.000561,0.002441,TO-2
VO-2,-0.026541,-0.030932,-0.042381,-0.028800,-0.016554,-1.000000,-0.005387,-0.003242,-0.065574,-0.007040,-0.003702,-0.000287,TO-2
PHC-1,-0.006911,-0.009927,-0.013203,-0.020534,-0.015974,-0.015155,-1.000000,-0.004742,-0.024206,-0.005253,-0.002163,0.002127,TO-2
PHC-2,-0.013686,-0.010650,-0.020718,-0.023550,-0.024504,-0.019061,-0.013068,-1.000000,-0.041063,-0.009380,-0.002641,0.002015,TO-2
LO-1,-0.025871,-0.022600,-0.038138,-0.041837,-0.039695,-0.034139,-0.025031,-0.020183,-1.000000,-0.014295,-0.001417,0.006512,TO-2
LO-2,-0.015462,-0.013838,-0.019509,-0.022416,-0.025884,-0.024249,-0.023129,-0.010786,-0.038118,-1.000000,0.016471,0.020284,TO-2


In [8]:
results_allP = np.mean(results_both_hemis, axis=0)
full_results = pd.DataFrame(results_allP, columns=rois.keys(), index=rois.keys())
full_results["best_roi"]  = full_results.idxmax(axis=1)
full_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.014308,0.012451,0.005573,0.000912,0.001520,-0.001586,-0.004401,-0.003277,-0.006977,-0.006356,-0.002538,-0.002037,V1
V2,0.016105,0.006763,0.004345,-0.001435,-0.000823,-0.002749,-0.003510,-0.001717,-0.010584,-0.005507,0.000047,-0.000613,V1
V3,0.010051,0.015139,0.005152,0.004061,0.000796,-0.000428,-0.002007,-0.000973,-0.008774,-0.004703,-0.002068,-0.000944,V2
hV4,0.004486,0.003941,-0.001402,-0.004630,-0.002560,-0.004948,-0.004961,-0.003685,-0.013277,-0.005008,-0.001228,-0.001463,V1
VO-1,0.006706,0.008151,0.003971,0.003001,0.001277,0.001363,0.000072,0.001936,-0.009735,-0.005695,-0.001103,0.000309,V2
VO-2,-0.000138,-0.000724,-0.006910,-0.008252,-0.003914,-0.004376,-0.002757,-0.000949,-0.022764,-0.006627,-0.000653,-0.001470,V1
PHC-1,0.001402,0.002448,0.001255,0.001692,0.000608,-0.000194,0.002503,0.004770,-0.007473,-0.004587,0.001462,0.001932,PHC-2
PHC-2,0.001687,0.002122,0.001079,0.004942,0.019582,0.022184,0.026418,0.031527,-0.001585,0.009905,0.034950,0.030565,TO-1
LO-1,-0.000896,-0.000247,-0.002711,-0.001803,0.010669,0.024115,0.026166,0.035203,-0.006522,0.009693,0.040010,0.037495,TO-1
LO-2,-0.001518,-0.000532,-0.001073,0.000948,0.013004,0.024397,0.032699,0.044865,0.000303,0.012750,0.043284,0.040157,PHC-2


In [31]:
# I think the low quality of P3 and P7 is messing up everything
results_both_hemis = np.delete(results_both_hemis, 2, axis=0)
results_both_hemis = np.delete(results_both_hemis, 5, axis=0)

In [32]:
results_both_hemis.shape

(6, 12, 12)

In [33]:
results_allP = np.mean(results_both_hemis, axis=0)
full_results = pd.DataFrame(results_allP, columns=rois.keys(), index=rois.keys())
full_results["best_roi"]  = full_results.idxmax(axis=1)
full_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.020963,0.016162,0.008179,0.002262,-0.000161,-0.002742,-0.005804,-0.004329,-0.006022,-0.007787,-0.003642,-0.003067,V1
V2,0.023804,-1.000000,0.008463,0.000542,-0.000781,-0.003120,-0.005173,-0.003237,-0.006692,-0.006478,-0.001151,-0.002017,V1
V3,0.015343,0.021402,-1.000000,0.008641,0.000890,-0.000440,-0.003037,-0.001735,-0.003814,-0.005293,-0.003174,-0.002408,V2
hV4,0.010757,0.011410,0.005999,-1.000000,-0.001294,-0.004841,-0.006489,-0.005020,-0.004261,-0.004875,-0.001704,-0.002565,V2
VO-1,0.009386,0.011861,0.008111,0.005966,-1.000000,0.001216,-0.000965,0.001465,-0.007011,-0.007055,-0.001944,-0.000337,V2
VO-2,0.008284,0.008306,0.004395,-0.001323,-0.000219,-1.000000,-0.002896,-0.000964,-0.009956,-0.005203,-0.000190,-0.002362,V2
PHC-1,0.003972,0.005419,0.006102,0.008513,0.003620,0.001686,-1.000000,0.005239,-0.002200,-0.004708,0.000925,0.001340,hV4
PHC-2,0.003825,0.003911,0.004417,0.009158,0.028415,0.029530,0.033307,-1.000000,0.004795,0.014041,0.042403,0.037715,TO-1
LO-1,0.003710,0.003745,0.003894,0.005046,0.020014,0.033698,0.032938,0.041722,-1.000000,0.014534,0.044976,0.041833,TO-1
LO-2,0.001290,0.001810,0.002616,0.005574,0.021383,0.032484,0.042445,0.053951,0.008421,-1.000000,0.047990,0.044549,PHC-2
